<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/KoGPT_2_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00


In [2]:
import transformers
transformers.__version__

'4.31.0'

In [4]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7c7aa77ed330>)

In [6]:
train_data=pd.read_table('ratings_train.txt')
test_data=pd.read_table('ratings_test.txt')

In [7]:
print('훈련용 리뷰 개수 : ', len(train_data))

훈련용 리뷰 개수 :  150000


In [8]:
print('테스트용 리뷰 개수 : ', len(test_data))

테스트용 리뷰 개수 :  50000


In [9]:
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [13]:
train_data.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [14]:
test_data.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [16]:
train_data.dropna(how='any', inplace=True)
train_data.reset_index(drop=True, inplace=True) #인덱스 열 제거
test_data.dropna(how='any', inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [18]:
print(len(train_data))
print(len(test_data))

149995
49997


In [59]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [60]:
print(tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031]


In [61]:
print(tokenizer.tokenize('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카', '리스', '마', '▁없는', '▁악', '역']


In [62]:
tokenizer.decode(tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

'보는내내 그대로 들어맞는 예측 카리스마 없는 악역'

In [63]:
for elem in tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'):
  print(tokenizer.decode(elem))

보는
내
내
그대로
들어
맞
는
예측
카
리스
마
없는
악
역


In [64]:
print(tokenizer.decode(3))

<pad>


In [65]:
max_seq_len=128

In [66]:
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길이 : ', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 :  128


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [67]:
print(tokenizer.encode('나의 이름은 김의인이다.'))
print(pad_sequences([tokenizer.convert_tokens_to_ids(tokenizer.tokenize('나의 이름은 김의인이다.'))], maxlen=max_seq_len, value=tokenizer.pad_token_id))

[19520, 11824, 9324, 8143, 9720, 9016]
[[    3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3 19520 11824  9324  8143  9720  9016]]


In [68]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
  input_ids, data_labels=[],[]

  for example, label in tqdm(zip(examples, labels), total=len(examples)):

    bos_token = [tokenizer.bos_token]
    eos_token=[tokenizer.eos_token]
    tokens=bos_token+tokenizer.tokenize(example)+eos_token
    input_id=tokenizer.convert_tokens_to_ids(tokens) #tokenizer.tokenize를 하고 convert_tokens_to_ids를 하면 encode와 같음
    input_id=pad_sequences([input_id], maxlen=max_seq_len, padding='post', value=tokenizer.pad_token_id)[0]
    #pad_sequences는 리스트만 가능, 리스트를 한 번 씌워주었으므로 [0]인덱스만 꺼내오면 원래의 패딩한 문장만 꺼내올 수 있음

    #input_id의 길이가 최대길이가 아닐 경우 에러 발생
    assert len(input_id)==max_seq_len, 'Error with input length {} vs {}'.format(len(input_id), max_seq_len)
    input_ids.append(input_id)
    data_labels.append(label)

  input_ids=np.array(input_ids, dtype=int) #원본이 변경되어도 함께 변경되지 않는다.
  data_labels=np.asarray(data_labels, dtype=np.int32) #원본이 변경될 경우 같이 변경된다. array와 구조는 같음

  return input_ids, data_labels

In [69]:
train_X, train_y=convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 149995/149995 [01:07<00:00, 2226.03it/s]


In [70]:
test_X, test_y=convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 49997/49997 [00:23<00:00, 2159.31it/s]


In [71]:
input_id=train_X[0]
label=train_y[0]
print('단어에 대한 정수 인코딩 : ', input_id)
print('각 인코딩의 길이 : ', len(input_id))
print('정수 인코딩 복원 : ', tokenizer.decode(input_id))
print('레이블 : ', label)

단어에 대한 정수 인코딩 :  [    1  9050  9267  7700  9705 23971 12870  8262  7055  7098  8084 48213
     1     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 :  128
정수 인코딩 복원 :  </s> 아 더빙.. 진짜 짜증나네요 목소리</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [73]:
model=TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [74]:
max_seq_len=128

In [79]:
input_ids_layers=tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs=model([input_ids_layers])

In [80]:
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [81]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tfgpt2_model/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model'")


In [82]:
print(outputs[1])

(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by 

In [83]:
print(outputs[0][:,-1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")


In [88]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
  def __init__(self,model_name):
    super(TFGPT2ForSequenceClassification, self).__init__()
    self.gpt=TFGPT2Model.from_pretrained(model_name, from_pt=True)
    self.dropout=tf.keras.layers.Dropout(0.2)
    self.classifier=tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),activation='sigmoid', name='classifier')

  def call(self, inputs):
    outputs=self.gpt(input_ids=inputs)
    cls_token=outputs[0][:,-1]
    cls_token=self.dropout(cls_token)
    prediction=self.classifier(cls_token)

    return prediction


In [89]:
#TPU 작동 코드
resolver=tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://'+os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [90]:
strategy=tf.distribute.experimental.TPUStrategy(resolver)

In [92]:
with strategy.scope():
  model=TFGPT2ForSequenceClassification('skt/kogpt2-base-v2')
  optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss=tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [93]:
model.fit(train_X, train_y, epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2
3750/3750 [==============================] - 466s 97ms/step - loss: 0.3888 - accuracy: 0.8281 - val_loss: 0.3007 - val_accuracy: 0.8724
Epoch 2/2
3750/3750 [==============================] - 324s 86ms/step - loss: 0.2920 - accuracy: 0.8769 - val_loss: 0.3190 - val_accuracy: 0.8686


In [94]:
results=model.evaluate(test_X, test_y, batch_size=1024)
print('test loss, test acc: ', results)

49/49 [==============================] - 21s 323ms/step - loss: 0.3260 - accuracy: 0.8636
test loss, test acc:  [0.3260112702846527, 0.8635917901992798]


In [98]:
def sentiment_predict(new_sentence):
  bos_token=[tokenizer.bos_token]
  eos_token=[tokenizer.eos_token]
  tokens=bos_token+tokenizer.tokenize(new_sentence)+eos_token
  input_id=tokenizer.convert_tokens_to_ids(tokens)
  input_id=pad_sequences([input_id],maxlen=max_seq_len, padding='post', value=tokenizer.pad_token_id)[0]
  input_id=np.array([input_id])
  score=model.predict(input_id)[0][0]

  if(score>0.5):
    print('{:.2f}% 확률로 긍정 리뷰입니다.\n'.format(score*100))
  else:
    print('{:.2f}% 확률로 부정 리뷰입니다.\n'.format((1-score)*100))

In [99]:
sentiment_predict('보던거라 계속 보고 있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에 ')

1/1 [==============================] - 11s 11s/step
97.25% 확률로 부정 리뷰입니다.



In [100]:
sentiment_predict("이 영화 존잼입니다 대박")

1/1 [==============================] - 1s 758ms/step
98.87% 확률로 긍정 리뷰입니다.



In [101]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

1/1 [==============================] - 1s 881ms/step
98.25% 확률로 긍정 리뷰입니다.



In [102]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

1/1 [==============================] - 1s 736ms/step
94.93% 확률로 부정 리뷰입니다.



In [103]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

1/1 [==============================] - 1s 740ms/step
97.89% 확률로 부정 리뷰입니다.



In [104]:
sentiment_predict('감독 뭐하는 놈이냐?')

1/1 [==============================] - 1s 852ms/step
98.59% 확률로 부정 리뷰입니다.



In [105]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

1/1 [==============================] - 1s 715ms/step
97.82% 확률로 긍정 리뷰입니다.

